# Projeto de Aprendizagem Automática II

## Procura de Exoplanetas no Espaço através da Emissão de Luz de Estrelas

### Importação de Bibliotecas

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns; sns.set()

from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report, precision_score, confusion_matrix, accuracy_score
from sklearn.preprocessing import normalize, StandardScaler
from sklearn import linear_model
from scipy import ndimage, fft

%matplotlib inline

%load_ext autoreload
%autoreload 2

### Carregamento dos Dados

O primeiro passo é o carregamento dos dados, de modo a treinar e, consequentemente, testar o modelo gerado.

In [3]:
treino = pd.read_csv("../../../../Dados/dados_treino.csv")
teste = pd.read_csv("../../../../Dados/dados_teste.csv")

Após o carregamento dos mesmos, é necessário dividir os registos em *labels* e *features*. Note-se que esta divisão deve ser efetuada tanto no conjunto de treino, como no de teste.

In [4]:
X_train = treino.loc[:, treino.columns != 'LABEL'].values
y_train = treino.LABEL.values

X_test = teste.loc[:, teste.columns != 'LABEL'].values
y_test = teste.LABEL.values

### Modelo

Tendo em conta os resultados obtidos com um classificador linear *SGD* no momento do teste das tranformações efetuadas aos dados para classificação, este tipo de modelo mostrou-se merecedor de um estudo extra. Assim, de modo a otimizar o mesmo, é necessário instanciar uma variável com o classificador pretendido.

In [23]:
model = linear_model.SGDClassifier(max_iter=1000)

De forma a otimizar os parâmetros do modelo, estes devem ser definidos na forma de dicionário. Neste caso, foi optado pelo teste de várias combinações, vendo qual a que melhores resultados traria ao problema.

In [25]:
grid_param = {'epsilon': [0.001, 0.01, 0.1, 1, 0.0001, 0.00001], 
              'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
              'loss': ['hinge', 'log', 'perceptron', 'squared_loss', 'huber'],
              'penalty': ['l2', 'l1', 'elasticnet'],
              'alpha': [0.001, 0.0005, 0.0001, 0.00005, 0.00001],
              'eta0': [0.1, 0.001, 0.0001, 0.00001],
              'shuffle': [True, False]}

Foi tirado proveito do método *GridSearchCV*, permitindo uma pesquisa em grelha dentro das combinações de parâmetros pretendidas, em que exista uma validação em cruz. Neste caso, foi optada por uma validação cruzada de 10 subconjuntos, sendo a métrica de *accuracy* a métrica a observar.

In [26]:
from sklearn.model_selection import GridSearchCV
gd_sr = GridSearchCV(estimator=model,
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=10,
                     n_jobs=-1,
                     verbose=2)

Tendo o modelo definido e pronto a efetuar a procura, apenas é necessário aplicar o método *fit* com o conjunto de dados de treino. Não faria sentido aplicar a validação cruzada aos dados de teste juntamente com os de treino, já que o modelo ficaria demasiado ajustado aos dados, perdendo a capacidade de classificar corretamente novos casos.

In [27]:
gd_sr.fit(X_train, y_train)

Fitting 10 folds for each of 14400 candidates, totalling 144000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   42.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 28.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 38.4min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 60.9min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 89.6min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 110.1min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 146.0min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 163.6min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 213.8min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed: 255.1min
[Parallel(n_jobs=-1)]: Done 6829 tasks      | elapsed: 314.4min
[Parallel(n_jobs=-1)]: Done 7922 tasks      | elapsed: 361.0min
[Parallel(n_jobs=-1)]: Done 9097 tasks  

GridSearchCV(cv=10, error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2', power_t=0.5,
                                     random_state=None, shuffle=True, tol=0.001,
                                     validation_fraction=0.1...
             param_grid={'alpha': [0.001, 0.0005, 0.0001, 5e-05, 1e-05],
                         'epsilon': [0.001, 0.01, 0.1, 1, 0.0001, 1e-05],
                         'eta0': [0.1, 0.001, 0.0001, 1e-05],
                         'learning_rate': ['constant', 'optimal', 'invscaling',
                            

Como se pode ver em seguida, os melhores parâmetros de entre os testados são *alpha* com valor 0.0001 e *epsilon* a 1. Já *eta0*, ou seja, o valor inicial para a taxa de aprendizagem, com valor de 0.00001, sendo a taxa de aprendizagem *invscaling*. Além disso, a *loss* do modelo é *huber*. O conjunto de treino foi baralhado a cada época e o termo de regularização (*penalty*) que melhores resultados trouxe foi *elasticnet*, podendo trazer alguma *feature selection* ao modelo, segundo a [documentação](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html).

In [28]:
best_parameters = gd_sr.best_params_
print(best_parameters)

{'alpha': 0.0001, 'epsilon': 1, 'eta0': 1e-05, 'learning_rate': 'invscaling', 'loss': 'huber', 'penalty': 'elasticnet', 'shuffle': True}


O melhor resultado obtido para a pesquisa em grelha, com os parâmetros acima mencionados, permitiu uma *accuracy* de sensivelmente 99.4%.

In [29]:
best_result = gd_sr.best_score_
print(best_result)

0.9939068474588856


### Teste

Conhecidos os melhores parâmetros para este modelo, é necessário ver de que forma se comporta na classificação do conjunto de teste.

In [30]:
preds = gd_sr.predict(X_test)

Observando as métricas obtidas pela predição dos registos de teste, é notório que que a precisão da classe minoritária subiu quando em comparação com o modelo original (tinha valor de 50%), mantendo-se o *recall*.

In [33]:
from sklearn import metrics
print(metrics.classification_report(y_test, preds))
print("accuracy:", metrics.accuracy_score(y_test, preds))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       565
           2       0.60      0.60      0.60         5

    accuracy                           0.99       570
   macro avg       0.80      0.80      0.80       570
weighted avg       0.99      0.99      0.99       570

accuracy: 0.9929824561403509


Observando a matriz de confusão e, mais uma vez, comparando com o modelo original, foram classificados 3 dos 5 sistemas da classe minoritária corretamente, sendo que agora, foi classificado de forma correta mais um sistema da classe maioritária, restando apenas 2 sistemas sem exoplanetas incorretamente classificados.

In [32]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[563   2]
 [  2   3]]


Em jeito de conclusão sobre este classificador, a otimização permitiu melhorar ligeiramente os resultados, na medida em que o modelo foi capaz de não classificar erradamente tantos registos como sendo da classe minritária como numa fase inicial. Apesar disso, o número de classificações corretas de sistemas com exoplanetas menteve-se, não sendo possível alcançar valores mais altos da métrica de *recall*. Assim, este tipo de classificador dificilmente permitirá obter melhores resultados do que os obtidos para os dados em estudo.